In [ ]:
body {
  padding: 0;
  margin: 0;
}

#map,
body,
html {
  height: 100%;
}


In [ ]:

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Earthquake Visualization</title>
  <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
  <link rel="stylesheet" href="style.css">
</head>
<body>
  <div id="map" style="height: 600px;"></div>

  <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
  <script src="https://d3js.org/d3.v5.min.js"></script>
  <script src="app.js"></script>
</body>
</html>

In [ ]:
body {
  font-family: Arial, sans-serif;
  margin: 0;
  padding: 0;
  background-color: #f0f0f0;
}

#map {
  width: 100%;
  height: 100vh;
}


In [ ]:
// Create the map object centered at a specific latitude and longitude with a zoom level
var map = L.map('map').setView([37.09, -95.71], 5); // Coordinates for the US center

// Add a TileLayer for the map background (you can change the tile layer for different styles)
L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
  attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
}).addTo(map);


In [ ]:
// URL for the USGS GeoJSON feed (Earthquakes from the past 7 days)
var geojsonURL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson";

// Fetch and load the data
d3.json(geojsonURL).then(function(data) {
  // Call the function to add the earthquake markers to the map
  createMarkers(data.features);
});


In [ ]:
function createMarkers(earthquakeData) {
    // Loop through each earthquake in the data
    earthquakeData.forEach(function(earthquake) {
      var coords = earthquake.geometry.coordinates;
      var magnitude = earthquake.properties.mag;
      var depth = coords[2];
      var location = earthquake.properties.place;
  
      // Calculate marker size based on magnitude
      var markerSize = magnitude * 5;
  
      // Define color based on depth (darker color for deeper earthquakes)
      var depthColor = getDepthColor(depth);
  
      // Create a circle marker at the earthquake's location
      L.circleMarker([coords[1], coords[0]], {
        radius: markerSize,
        fillColor: depthColor,
        color: "#000",
        weight: 0.5,
        opacity: 1,
        fillOpacity: 0.7
      })
      .bindPopup("<h3>Location: " + location + "</h3><p>Magnitude: " + magnitude + "<br>Depth: " + depth + " km</p>")
      .addTo(map);
    });
  }
  
  // Function to determine color based on depth
  function getDepthColor(depth) {
    if (depth <= 10) return "#00FF00"; // Shallow
    else if (depth <= 30) return "#FFFF00"; // Moderate
    else if (depth <= 50) return "#FFA500"; // Deep
    else return "#FF0000"; // Very deep
  }
  

In [ ]:
// Create the legend to show the color scale for depth
var legend = L.control({position: 'bottomright'});

legend.onAdd = function() {
  var div = L.DomUtil.create('div', 'info legend');
  var depthRanges = [0, 10, 30, 50];
  var colors = ["#00FF00", "#FFFF00", "#FFA500", "#FF0000"];

  // Loop through the depth ranges and colors to build the legend
  for (var i = 0; i < depthRanges.length; i++) {
    div.innerHTML +=
      '<i style="background:' + colors[i] + '"></i> ' +
      depthRanges[i] + (depthRanges[i + 1] ? '&ndash;' + depthRanges[i + 1] : '+') + ' km<br>';
  }
  return div;
};

legend.addTo(map);
